Import Library

In [ ]:
from PIL import Image
import os
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

1. Data Preprocessing

In [ ]:
def zoom_out(image_path, zoom_factor):
    # Ambil nama gambar
    imgWithFormat = os.path.basename(image_path)
    imgName = imgWithFormat.split(".")[0]
    
    # Buka gambar 
    original_image = Image.open(image_path)
    original_width, original_height = original_image.size

    # Hitung ukuran baru setelah diperbesar
    new_width = int(original_width * zoom_factor)
    new_height = int(original_height * zoom_factor)
    
    # Perbesar gambar menggunakan metode resampling lanczos
    zoomed_out_image = original_image.resize((new_width, new_height), Image.LANCZOS)

    # Buat latar belakang hitam dengan ukuran sesuai gambar asli
    black_background = Image.new('RGB', (original_width, original_height), (0, 0, 0))

    # Hitung posisi gambar yang diperbesar di tengah latar belakang hitam
    x_offset = (original_width - new_width) // 2
    y_offset = (original_height - new_height) // 2
    
    # Tempelkan gambar yang diperbesar ke latar belakang hitam
    black_background.paste(zoomed_out_image, (x_offset, y_offset))

    return black_background
    
def convert_to_binary(image : np.ndarray, threshold=128):
    _, binary_image = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)
    
    # # Ubah ke gambar grayscale
    # image = cv2.cvtColor(binary_image, cv2.COLOR_BGR2GRAY)
    
    # Simpan gambar biner
    return binary_image

def resize_image(image : np.ndarray, px : int): 
    return cv2.resize(image, (px, px))


2. Zoom Out semua Gambar

In [ ]:
def zoom_out_image_in_folder(path): 
    zoom_images = []
    
    for folder_name in os.listdir(path): # Loop nama folder
        folder_path = os.path.join(path, folder_name)
        
        for file_name in os.listdir(folder_path): 
            file_path = os.path.join(folder_path, file_name)
            
            zoom_factors = [0.5, 0.6, 0.7, 0.8, 0.9, 1]
            
            img_name = file_name.split('.')[0]
            
            for factor in zoom_factors:
                # Lakukan zoom out
                zoomed_image = zoom_out(file_path, factor)
                
                np_image = np.array(zoomed_image) 
                
                 # Tambahkan nama folder sebagai target
                image_list = {
                    "id": folder_name,
                    "image": np_image
                }
                
                # Tambahkan gambar ke list
                zoom_images.append(image_list)

    return zoom_images

for i in tqdm(range(100)):
    zoomed_images = zoom_out_image_in_folder("dataset")

In [ ]:
# Ambil 5 gambar acak dari array zoomed_images
random_items = np.random.choice(zoomed_images, size=5, replace=False)

# Tampilkan 5 gambar acak menggunakan plt.imshow()
for i, item in enumerate(random_items, start=1):
    img_id = item["id"]
    img = item["image"]
    
    # Tampilkan gambar menggunakan plt.imshow()
    plt.subplot(1, 5, i)
    plt.imshow(img)
    plt.title(img_id)  # Tambahkan judul sesuai dengan nama folder
    plt.axis('off')  # Matikan sumbu x dan y

plt.tight_layout()
plt.show()

Resize Gambar 

In [ ]:
def resize_all_images(image : list):
    resized_image = []
    
    pixel = 25
    
    for item in image: 
        image = resize_image(item["image"], pixel)
        
        image_list = {
            "id": item["id"],
            "image": image
        }
        
        resized_image.append(image_list)
    
    return resized_image
    
resized_img = resize_all_images(zoomed_images)
        
random_items = np.random.choice(resized_img, size=5, replace=False)

# Tampilkan 5 gambar acak menggunakan plt.imshow()
for i, item in enumerate(random_items, start=1):
    img_id = item["id"]
    img = item["image"]
    
    # Tampilkan gambar menggunakan plt.imshow()
    plt.subplot(1, 5, i)
    plt.imshow(img)
    plt.title(img_id)  # Tambahkan judul sesuai dengan nama folder
    plt.axis('off')  # Matikan sumbu x dan y

plt.tight_layout()
plt.show()

Ubah Semua Gambar ke Bentuk Biner

In [ ]:
def binaried_all_images(image : list):
    binary_image = []
    
    for item in image: 
        image = convert_to_binary(item["image"], 80)
        
        image_list = {
            "id": item["id"],
            "image": image
        }
        
        binary_image.append(image_list)
        
    return binary_image

binaried_image = binaried_all_images(resized_img)

random_items = np.random.choice(binaried_image, size=5, replace=False)

# Tampilkan 5 gambar acak menggunakan plt.imshow()
for i, item in enumerate(random_items, start=1):
    img_id = item["id"]
    img = item["image"]
    
    # Tampilkan gambar menggunakan plt.imshow()
    plt.subplot(1, 5, i)
    plt.imshow(img)
    plt.title(img_id)  # Tambahkan judul sesuai dengan nama folder
    plt.axis('off')  # Matikan sumbu x dan y

plt.tight_layout()
plt.show()
    

Export Gambar Hasil Preprocessing

In [ ]:
def save_image(item, index, dst_folder):
    if not os.path.exists(dst_folder):
        os.makedirs(dst_folder)
    
    img_id = item["id"]
    
    # Konversi numpy image ke dalam format yang dapat dibaca OpenCV
    image = cv2.cvtColor(item["image"], cv2.COLOR_RGB2BGR)
    
    # Simpan gambar dalam format JPEG
    cv2.imwrite(f'{dst_folder}/{img_id}_{str(index)}.jpg', image)

for i in tqdm(range(100)):
    for index, item in enumerate(binaried_image):
        save_image(item, index, 'binary_image2')

Ekstraksi Fitur Dengan HOG

In [ ]:
def extract_hog_features(folder_path):
    labels = []
    features = []
    
    for filename in os.listdir(folder_path):
        # Baca gambar biner
        image_path = os.path.join(folder_path, filename)
        
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        px = 5
        # Hitung HOG descriptor
        feature = hog(image, orientations=9, pixels_per_cell=(px, px),
                      cells_per_block=(2, 2), block_norm='L2-Hys')
        
        label = filename.split("_")[0]
        
        labels.append(label)
        features.append(feature)
        
    return labels, features

labels, features = extract_hog_features('binary_image2')

print(f"Label: {labels}")
print(f"Jumlah Data: {len(labels)}")
print(f"Feature: {features}")
        
        



Membagi Data Menjadi Data Latih dan Data Uji

In [ ]:
from sklearn.model_selection import train_test_split

# Membagi dataset menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

Classifier Menggunakan K-Nearest Neighbors (KNN)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Membuat model KNN
knn_model = KNeighborsClassifier(n_neighbors=5)

# Melatih model KNN
knn_model.fit(X_train, y_train)

# Melakukan prediksi menggunakan data uji
y_pred = knn_model.predict(X_test)

# Mengukur akurasi model
accuracy = accuracy_score(y_test, y_pred)
print("Akurasi model KNN:", accuracy)


Classifier Menggunakan Adabost

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

# Membagi dataset menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Membuat model AdaBoost dengan DecisionTreeClassifier sebagai base estimator
adaboost_model = AdaBoostClassifier(n_estimators=20, random_state=42)

# Melatih model AdaBoost
adaboost_model.fit(X_train, y_train)

# Melakukan prediksi menggunakan data uji
y_pred = adaboost_model.predict(X_test)

# Mengukur akurasi model
accuracy = accuracy_score(y_test, y_pred)
print("Akurasi model AdaBoost:", accuracy)


Classifier Menggunakan Support Vector Machine (SVM) Kernel Linear

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# List nilai C yang akan diuji
list_C = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

best_accuracy = 0
best_C = None

# Loop melalui setiap nilai C
for C in list_C:
    # Membuat model SVM dengan kernel linear dan nilai C tertentu
    svm_linear = SVC(kernel='linear', C=C, random_state=42)
    
    # Melatih model SVM
    svm_linear.fit(X_train, y_train)
    
    # Melakukan prediksi menggunakan data uji
    y_pred = svm_linear.predict(X_test)
    
    # Mengukur akurasi model
    accuracy = accuracy_score(y_test, y_pred)
    
    # Memperbarui akurasi tertinggi dan nilai C terbaik jika ditemukan akurasi yang lebih tinggi
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_C = C


In [ ]:
print(f"best_accuracy - best_C: {best_accuracy} - {best_C}")

In [ ]:
# Membuat model SVM
svm_linear = SVC(kernel='linear', C=7, random_state=42)

# Melatih model SVM
svm_linear.fit(X_train, y_train)

# Melakukan prediksi menggunakan data uji
y_pred = svm_linear.predict(X_test)

# Mengukur akurasi model
accuracy = accuracy_score(y_test, y_pred)
print("Akurasi model SVM dengan kernel linear:", accuracy)

Classifier Menggunakan Support Vector Machine (SVM) Kernel Polynomial

In [ ]:
# List nilai C yang akan diuji
list_degree = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

best_accuracy = 0
best_degree = None

# Loop melalui setiap nilai C
for D in list_degree:
    # Membuat model SVM dengan kernel polinomial
    svm_poly = SVC(kernel='poly', degree=D)
    
    # Melatih model SVM dengan kernel polinomial
    svm_poly.fit(X_train, y_train)

    # Melakukan prediksi menggunakan data uji
    y_pred_poly = svm_poly.predict(X_test)

    # Mengukur akurasi model
    accuracy = accuracy_score(y_test, y_pred_poly)
    
    # Memperbarui akurasi tertinggi dan nilai C terbaik jika ditemukan akurasi yang lebih tinggi
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_degree = D
        
    print(f"best_accuracy :{best_accuracy}\nbest_degree :{best_degree}")

In [ ]:
# Membuat model SVM dengan kernel polinomial
svm_poly = SVC(kernel='poly', degree=4)

# Melatih model SVM dengan kernel polinomial
svm_poly.fit(X_train, y_train)

# Melakukan prediksi menggunakan data uji
y_pred_poly = svm_poly.predict(X_test)

# Mengukur akurasi model
accuracy_poly = accuracy_score(y_test, y_pred_poly)
print("Akurasi model SVM dengan kernel polinomial:", accuracy_poly)


Classifier Menggunakan Support Vector Machine (SVM) Kernel RBF

In [ ]:
# Membuat model SVM dengan kernel RBF
svm_rbf = SVC(kernel='rbf')

# Melatih model SVM dengan kernel RBF
svm_rbf.fit(X_train, y_train)

# Melakukan prediksi menggunakan data uji
y_pred_rbf = svm_rbf.predict(X_test)

# Mengukur akurasi model
accuracy_rbf = accuracy_score(y_test, y_pred_rbf)
print("Akurasi model SVM dengan kernel RBF:", accuracy_rbf)

Classifier Menggunakan Support Vector Machine (SVM) Kernel Sigmoid

In [ ]:
# Membuat model SVM dengan kernel sigmoid
svm_sigmoid = SVC(kernel='sigmoid')

# Melatih model SVM dengan kernel sigmoid
svm_sigmoid.fit(X_train, y_train)

# Melakukan prediksi menggunakan data uji
y_pred_sigmoid = svm_sigmoid.predict(X_test)

# Mengukur akurasi model
accuracy_sigmoid = accuracy_score(y_test, y_pred_sigmoid)
print("Akurasi model SVM dengan kernel sigmoid:", accuracy_sigmoid)

In [ ]:
import joblib

# # Menyimpan model KNN
# joblib.dump(knn_model, 'model_knn.pkl')

# # Menyimpan model SVM dengan kernel linear
# joblib.dump(svm_linear, 'svm_linear.pkl')

# Menyimpan model SVM dengan kernel polinomial
joblib.dump(svm_poly, 'svm_poly.pkl')

# # Menyimpan model SVM dengan kernel RBF
# joblib.dump(svm_rbf, 'svm_rbf.pkl')

# # Menyimpan model SVM dengan kernel sigmoid
# joblib.dump(svm_sigmoid, 'svm_sigmoid.pkl')